In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [3]:
d = DataLoader('AAPL', '2018-01-01', '2019-12-31')

In [8]:
prices = d.data.iloc[0]

In [26]:
def price_df(price, period=390):
    # Create datetime index by the minutes
    # Start time : 9:30AM
    # 390 minuntes per trading day
    ts = price.name
    ts = ts.replace(hour=9, minute=31)
    index = pd.date_range(ts, periods=period, freq="1min")

    # Stacking all price array
    array = np.stack([np.zeros(period) + price['Open'],
                      np.zeros(period) + price['High'],
                      np.zeros(period) + price['Low'],
                      np.zeros(period) + price['Close']
                      ]).T
    return pd.DataFrame(array, index=index, columns=['Open', 'High', 'Low', 'Close'])

In [66]:
period = 390
# High and Low indice
close_to_low = prices['Close'] / prices['Low'] - 1
open_to_low = prices['Open'] / prices['Low'] - 1
swap = True if open_to_low < close_to_low else False
reduce = np.random.choice([int(period * .40), int(period * .60)]) 
if swap:
    low_i = np.random.choice(np.arange(1, reduce))
    high_i = np.random.choice(np.arange(low_i, period - 1))
else:
    high_i = np.random.choice(np.arange(1, reduce))
    low_i = np.random.choice(np.arange(high_i, period - 1))
print(swap, low_i, high_i)

True 174 371


In [68]:
# Creating psudo Low prices
low_price = np.zeros(period) + prices['Low']

array([1.02794515, 1.01237307, 1.04667569, 1.01041353, 1.02350066,
       1.01182512, 0.96482267, 0.99408847, 0.96874629, 0.95273376,
       0.9551492 , 1.04763106, 1.03377929, 1.01625469, 1.03826422,
       0.97387879, 0.97795306, 0.96487686, 0.97383329, 0.95794386,
       1.0148798 , 0.97852308, 0.97564787, 0.99513792, 0.9962947 ,
       1.04473235, 0.9705741 , 0.96390832, 0.96054642, 1.02952427,
       1.03013555, 0.9930244 , 0.96481437, 0.98349758, 1.02300231,
       1.00462456, 0.96340611, 0.97353559, 1.04394521, 1.00466038,
       1.02494251, 0.95495662, 0.96068994, 1.03196572, 0.96900582,
       0.99179561, 1.04267758, 0.99916258, 1.00772946, 0.99083765,
       1.04880836, 0.96345834, 1.01008722, 0.98480663, 0.97681279,
       0.97916317, 0.97814691, 0.97192335, 0.97328632, 1.01230859,
       1.00531988, 0.95949827, 0.96123487, 0.99732209, 0.98832554,
       0.98730497, 0.96915894, 1.03218862, 0.9834056 , 0.99848615,
       1.02696354, 1.0338754 , 0.96595596, 1.02872603, 1.04944